In [39]:
from my_utils import tile_processing as tp

from stardist.matching import matching_dataset as scorer
import os
from tqdm import tqdm
import pandas as pd
from scipy.io import loadmat

In [40]:
model_name = 'HN_Kumar'
base_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets"
path_ints = os.path.join(base_folder, "Interpolations")
path_segs = os.path.join(base_folder, "Segmentations " + model_name)
interpolations = os.listdir(path_ints)

In [41]:
taus = [0.5]
columns = ['Model', 'Interpolation', 'Dataset', 'Tile', 'Tau', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1-Score', 'Panoptic Quality']
df_scores = pd.DataFrame(columns=columns)

In [42]:
def write_row(column_names, vals, df):
    row = pd.DataFrame(pd.Series(index=column_names)).T
    for c, col_name in enumerate(column_names):
        row[col_name] = vals[c]
    return pd.concat([df, row], ignore_index=True, axis=0)

In [43]:
for interpolation in tqdm(interpolations, desc='Interpolation'):
# for interpolation in [interpolations[0]]:
    path_datasets = os.path.join(path_ints, interpolation)
    path_datasets_HN = os.path.join(path_segs, interpolation)
    for dataset in os.listdir(path_datasets):
    # for dataset in [os.listdir(path_datasets)[0]]:
        path_gts = os.path.join(os.path.join(path_datasets, dataset), r'masks')
        path_preds = os.path.join(os.path.join(path_datasets_HN, dataset), r'images\mat')
        # pred = loadmat('your_file.mat')['inst_map']
        tile_set = tp.TileSetReader(path_gts, '.tif').tile_set
        for i, tilename in enumerate(tile_set[0]):
            gt = tile_set[1][i]
            pred = loadmat(os.path.join(path_preds, tilename + '.mat'))['inst_map']
            for tau in taus:
                scores = scorer([gt], [pred], thresh=tau, show_progress=False)._asdict()
                values = [model_name, interpolation, dataset, tilename, tau,
                          scores['tp'], scores['fp'], scores['fn'],
                          scores['precision'], scores['recall'], scores['f1'], scores['panoptic_quality']]
                df_scores = write_row(columns, values, df_scores)

Interpolation: 100%|██████████| 26/26 [04:31<00:00, 10.43s/it]


In [44]:
df_scores

,Model,Interpolation,Dataset,Tile,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,HN_Kumar,L - 20x - 0_,CoNSeP,test_1,0.5,111,21,753,0.840909,0.128472,0.222892,0.165767
1,HN_Kumar,L - 20x - 0_,CoNSeP,test_10,0.5,176,87,669,0.669202,0.208284,0.317690,0.234297
2,HN_Kumar,L - 20x - 0_,CoNSeP,test_11,0.5,161,144,916,0.527869,0.149489,0.232996,0.165535
3,HN_Kumar,L - 20x - 0_,CoNSeP,test_12,0.5,33,9,395,0.785714,0.077103,0.140426,0.099562
4,HN_Kumar,L - 20x - 0_,CoNSeP,test_13,0.5,253,103,1106,0.710674,0.186166,0.295044,0.224156
...,...,...,...,...,...,...,...,...,...,...,...,...
2621,HN_Kumar,R - 40x - 3_,JHU,Skin_train_1,0.5,182,36,59,0.834862,0.755187,0.793028,0.618698
2622,HN_Kumar,R - 40x - 3_,JHU,Skin_train_2,0.5,45,33,40,0.576923,0.529412,0.552147,0.416972
2623,HN_Kumar,R - 40x - 3_,JHU,Skin_train_3,0.5,447,107,366,0.806859,0.549815,0.653987,0.495921
2624,HN_Kumar,R - 40x - 3_,JHU,Skin_train_4,0.5,229,36,167,0.864151,0.578283,0.692890,0.549404


In [45]:
out_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Scores on Interpolations"
df_scores.to_csv(os.path.join(out_folder, f'{model_name}.csv'), index=False)